# Задание
1) Задача классификации изображений (https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html?highlight=mnist). Повторить тренировку модели (train) и запустить классификацию изображений (inference). 

2) Получить максимальную точность классификации (минимальный loss) путём изменения модели, например, добавлением скрытых слоёв

In [15]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
#нашла этот кусок в интернетах, без него умирает kernel :(

In [16]:
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch
import torchvision
import torchvision.transforms as transforms
#библиотеки

In [17]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5),
                                                     (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=50,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=50,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')
#загружаем датасет, называем классы

Files already downloaded and verified
Files already downloaded and verified


In [18]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
                    nn.Conv2d(in_channels=3, out_channels=6, kernel_size=3, padding=3),
                    nn.ReLU())
        
        
        self.layer2 = nn.Sequential(
                    nn.Conv2d(in_channels=6, out_channels=16, kernel_size=3, padding=1),
                    nn.ReLU(),
                    nn.MaxPool2d(2, 2))
        
        self.layer3 = nn.Sequential(
                    nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=3, dilation=3),
                    nn.ReLU(),
                    nn.MaxPool2d(2,2))
        
        self.layer4 = nn.Sequential(
                    nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=3, dilation=3),
                    nn.ReLU(),
                    nn.MaxPool2d(2,2))
        
        self.layer5 = nn.Sequential(
                    nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=3, dilation=3),
                    nn.ReLU(),
                    nn.MaxPool2d(2,2))
        
        self.layer6 = nn.Sequential(
                    nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=3, dilation=3),
                    nn.ReLU(),
                    nn.MaxPool2d(2,2))
    
        
        self.fc1 = nn.Linear(256, 150)
        self.fc2 = nn.Linear(150, 50)
        self.fc3 = nn.Linear(50, 10)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer6(x)
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [19]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
#кросс-энтропия и оптимизатор адам

In [20]:
 for epoch in range(15):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 1000 == 999:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 1000))
            running_loss = 0.0

print('Finished Training')
#обучение

[1,  1000] loss: 1.807
[2,  1000] loss: 1.486
[3,  1000] loss: 1.309
[4,  1000] loss: 1.180
[5,  1000] loss: 1.084
[6,  1000] loss: 1.008
[7,  1000] loss: 0.942
[8,  1000] loss: 0.885
[9,  1000] loss: 0.830
[10,  1000] loss: 0.785
[11,  1000] loss: 0.739
[12,  1000] loss: 0.691
[13,  1000] loss: 0.650
[14,  1000] loss: 0.615
[15,  1000] loss: 0.584
Finished Training


In [23]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' %
      (100 * correct / total))

Accuracy of the network on the 10000 test images: 64 %


In [24]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane : 78 %
Accuracy of   car : 77 %
Accuracy of  bird : 62 %
Accuracy of   cat : 39 %
Accuracy of  deer : 56 %
Accuracy of   dog : 46 %
Accuracy of  frog : 74 %
Accuracy of horse : 62 %
Accuracy of  ship : 82 %
Accuracy of truck : 62 %
